## **TurboBatch User Dashboard**

### **Environment Set-up**
Before submitting your job please ensure you have created a new Google Cloud Project with an associated billing account and have the gcloud CLI working in your environment. Additionally, this dashboard requires certain packages be installed. Please make sure to `pip install -r requirements.txt` in your environment before continuing.

The scripts below will assist with any further steps:

**Ensure our system's service account has all required permissions to your project**

To do this, you can simply run our privileges.sh bash script in the terminal. Please type the following command, replacing [project-id] with your actual project's ID. Simple select 'yes' when prompted and complete the authorization process on the browser.

```./privileges.sh [project-id]```

### **Upload Input DataFrame into BigQuery Table**

Our system requires an input in the form of a BigQuery Table and writes to an output table in the same form. To streamline the process of the creation of these tables, simply run the cell below after changing the variables to upload and create an input table along with creating an empty output table in your project.

Please note that the input table must be formated with two columns: `row` (integers) and `prompt_and_text` (strings). See `example.csv` for reference.



In [ ]:
from helpers.df import CSV_to_BigQuery

#TODO Change the below variables before running this cell
input_CSV_path = "example.csv"
project_id = "sampleproject-440900"
dataset_id = "test"
input_table_id = "input"
output_table_id = "output"

CSV_to_BigQuery(input_CSV_path, project_id, dataset_id, input_table_id, output_table_id)

### **Submit Job**

Fill in the relevant information below and then run the cell to submit a job.

Note:
- Currently supported model: "gpt-3.5-turbo"
- Leaving API_key empty ("") will result in "sample_response" being returned for all input data.

In [ ]:

#TODO Change the below variables before running this cell
user_data = {
    "username": "test_user",
    "password": "test_password",
    "Client_ID": "test",
    "User_Project_ID": "sampleproject-440900",
    "User_Dataset_ID": "test",
    "Input_Table_ID": "input",
    "Output_Table_ID": "output",
    "Model": "gpt-3.5-turbo",
    "API_key": ""  
}

import logging
import json
from helpers.jobPrep import retrieve_api_key, get_job_data, submit_job
from helpers.jobStatus import wait_for_completion

# Step 1: Retrieve API Key
logging.info("🔑 Retrieving API key...")
api_key = retrieve_api_key(user_data)

# Step 2: Prepare and Submit Job
if api_key:
    logging.info("📋 Preparing job data...")
    job_data = get_job_data(user_data)
    logging.info("📤 Submitting job...")
    job_id = submit_job(api_key, job_data)

    # Step 3: Monitor Job Status
    if job_id:
        logging.info("⏳ Monitoring job status...")
        final_status = wait_for_completion(job_id, user_data["Client_ID"], api_key,3)
        if final_status:
            logging.info("📊 Final Job Status Retrieved:")
            logging.info(json.dumps(final_status, indent=4))

### **Manually Check Job Status by Running Cell Below**

If you would like to check on your job status please run the cell below to do so. Note that this can only be run after the previous cell has been run.

In [ ]:
from helpers.jobStatus import check_job_status

check_job_status(job_id=job_id, client_id=user_data["Client_ID"], api_key=api_key)

### **Save Output BigQuery Table as a CSV**

If you prefer to have the output responses and corresponding rows saved in your local machine as a CSV rather than checking the BigQuery table in your project, run the cell below to access the output table.

In [ ]:
from helpers.df import BigQuery_to_CSV

#TODO Change the below variables before running this cell
project_id = "sampleproject-440900"
dataset_id = "test"
output_table_id = "output"
output_csv_path = "output.csv"

BigQuery_to_CSV(project_id, dataset_id, output_table_id, output_csv_path)